<br>
<h1 style = "font-size:40px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">PCW: Learning How to Use Folium with Crime Data in SF</h1>
<br>
    

### <h3 style="color:#fe346e">Overview</h3>

This PCW consists of two parts.
1. **Tutorial: Create Interactive Map with the Folium package ~Boston~**
- For the first part (Boston example), the code is already completed. Thus,your task is go through each cell and understand what the code is doing and how it is working. You can add in-line comment. There are several questions listed in your way, so be ready to share your answer at the class.
2. **Your Turn: Create Interactive Map with the Folium package ~San Francisco~**
- For the second part (San Francisco example), you will do very similar things with part 1, but some of the code is missing. Please fill the code by yourself. 




<html><h3 style='font-family:Garamond;background:lightgrey;  color:black; font-size:30px; padding:15px; border:3px solid #fe346e;'><center><b> Tutorial: Create Interactive Map with the Folium package ~Boston~ </b></center></h3></html>

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #ffffff; color: navy;">Import Libraries&nbsp;&nbsp;&nbsp;&nbsp;</h1> 

In [40]:
import pandas as pd
import geopandas as gpd
import math
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #ffffff; color: navy;">Display Boston Map with folium.Map() &nbsp;&nbsp;&nbsp;&nbsp;</h1> 

In [41]:
# Create a map
boston_map = folium.Map(location=[42.32,-71.0589], tiles='openstreetmap', zoom_start=10)

# Display the map
boston_map

Several arguments customize the appearance of the map:
- `location` sets the initial center of the map. We use the latitude (42.32&deg; N) and longitude (-71.0589&deg; E) of the city of Boston.  
- `tiles` changes the styling of the map; in this case, we choose the [OpenStreetMap](https://www.openstreetmap.org/#map=10/42.32/-71.0589) style.  If you're curious, you can find the other options listed [here](https://github.com/python-visualization/folium/tree/master/folium/templates/tiles).We can change this to Stamen Toner, CartoDB positron, Cartodb dark_matter, Stamen Watercolor or Stamen Terrain. 
- `zoom_start` sets the initial level of zoom of the map, where higher values zoom in closer to the map.


- Q: What is "zoom_start" ? 
- Q: How should I adjust "zoom_start" if I would like to make my map zoomed in?

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #ffffff; color: navy;">Import Data &nbsp;&nbsp;&nbsp;&nbsp;</h1> 

For the sake of data size, **df_boston.csv** only contains the crime information under "Robbery" category.

In [42]:
df_boston = pd.read_csv("df_boston.csv")
df_boston.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I182079671,301,Robbery,ROBBERY - STREET,B3,431,NaN,2018-10-02 16:57:00,2018,10,Tuesday,16,Part One,BLUE HILL AVE,42.288633,-71.089904,"(42.28863337, -71.08990388)"
1,I182079520,301,Robbery,ROBBERY - STREET,B2,280,NaN,2018-10-02 08:05:00,2018,10,Tuesday,8,Part One,DUDLEY ST,42.328632,-71.083099,"(42.32863172, -71.08309899)"
2,I182079510,301,Robbery,ROBBERY - STREET,C11,403,NaN,2018-10-02 06:40:00,2018,10,Tuesday,6,Part One,FULLER ST,42.282481,-71.069812,"(42.28248068, -71.06981187)"
3,I182079503,301,Robbery,ROBBERY - STREET,C11,253,NaN,2018-10-02 06:18:00,2018,10,Tuesday,6,Part One,BEACH ST,42.301789,-71.055142,"(42.30178949, -71.05514183)"
4,I182079502,381,Robbery,ROBBERY - CAR JACKING,C11,353,NaN,2018-10-02 00:00:00,2018,10,Tuesday,0,Part One,MILL ST,42.299211,-71.052038,"(42.29921088, -71.05203766)"


To reduce the amount of data we need to fit on the map, we'll (temporarily) confine our attention to daytime robberies.

In [43]:
daytime_robberies = df_boston[df_boston.HOUR.isin(range(9,18))]
daytime_robberies.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I182079671,301,Robbery,ROBBERY - STREET,B3,431,NaN,2018-10-02 16:57:00,2018,10,Tuesday,16,Part One,BLUE HILL AVE,42.288633,-71.089904,"(42.28863337, -71.08990388)"
8,I182079383,301,Robbery,ROBBERY - STREET,C11,351,NaN,2018-10-01 16:19:00,2018,10,Monday,16,Part One,CHARLES ST,42.300785,-71.060007,"(42.30078455, -71.06000691)"
10,I182079048,301,Robbery,ROBBERY - STREET,B2,262,NaN,2018-09-30 15:01:00,2018,9,Sunday,15,Part One,DUDLEY ST,42.321970,-71.072425,"(42.32196970, -71.07242529)"
19,I182078574,301,Robbery,ROBBERY - STREET,D4,165,NaN,2018-09-11 09:40:00,2018,9,Tuesday,9,Part One,HARRISON AVE,42.342850,-71.065162,"(42.34285014, -71.06516235)"
20,I182078527,301,Robbery,ROBBERY - STREET,B2,327,NaN,2018-09-28 14:09:00,2018,9,Friday,14,Part One,HARTFORD ST,42.317454,-71.073279,"(42.31745360, -71.07327858)"


Q: What is the column name for latitude and longtitude?

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #ffffff; color: navy;">Map1: folium.Marker()  &nbsp;&nbsp;&nbsp;&nbsp;</h1> 

We add markers to the map with folium.Marker(). Each marker below corresponds to a different robbery.

In [44]:
# Create a map
map1_boston = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=13)

# Add points to the map
for idx, row in daytime_robberies.iterrows():
    Marker([row['Lat'], row['Long']]).add_to(map1_boston)

# Display the map
map1_boston

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #ffffff; color: navy;">Map2: folium.plugins.MarkerCluster()  &nbsp;&nbsp;&nbsp;&nbsp;</h1> 

If we have a lot of markers to add, `folium.plugins.MarkerCluster()` can help to declutter the map.  Each marker is added to a `MarkerCluster` object.

In [45]:
# Create the map
map2_boston = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=13)

# Add points to the map
mc = MarkerCluster()
for idx, row in daytime_robberies.iterrows():
    if not math.isnan(row['Long']) and not math.isnan(row['Lat']):
        mc.add_child(Marker([row['Lat'], row['Long']]))
map2_boston.add_child(mc)

# Display the map
map2_boston

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #ffffff; color: navy;">Map3: Heatmap with folium.plugins.HeatMap()  &nbsp;&nbsp;&nbsp;&nbsp;</h1> 

 This shows the density of crime in different areas of the city, where yellow areas have relatively more criminal incidents.

In [46]:
# Create a base map
map3_boston = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=12)

# Add a heatmap to the base map
HeatMap(data=daytime_robberies[['Lat', 'Long']], radius=10).add_to(map3_boston)

# Display the map
map3_boston

What are the Pros and Cons of each graph?

Map2(Marker):
- Marker() function to mark the exact location of the coordinates. 
Map2(MarkerCluster): 
- It is a form of clustering so you can immediately assess which areas are populated with your object of study.
- it helps keep your map clean looking as it does not clutter it with too many markers.

<html><h3 style='font-family:Garamond;background:lightgrey;  color:black; font-size:30px; padding:15px; border:3px solid #fe346e;'><center><b> Your Turn: Create Interactive Map with the Folium package ~San Francisco~ </b></center></h3></html>

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #ffffff; color: navy;">Display San Francisco Map with folium.Map() &nbsp;&nbsp;&nbsp;&nbsp;</h1> 

In [47]:
# Create a map
sf_map = folium.Map(location=[37.78363773158004, -122.40941186660926], tiles='openstreetmap', zoom_start=10)

# Display the map
sf_map

Several arguments customize the appearance of the map:
- `location` sets the initial center of the map. We use the latitude (42.32&deg; N) and longitude (-71.0589&deg; E) of the city of Boston.  
- `tiles` changes the styling of the map; in this case, we choose the [OpenStreetMap](https://www.openstreetmap.org/#map=10/42.32/-71.0589) style.  If you're curious, you can find the other options listed [here](https://github.com/python-visualization/folium/tree/master/folium/templates/tiles).
- `zoom_start` sets the initial level of zoom of the map, where higher values zoom in closer to the map.


<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #ffffff; color: navy;">Import Data &nbsp;&nbsp;&nbsp;&nbsp;</h1> 

For the sake of data size, **df_sf_robbery.csv** only contains the crime information under **"Robbery"** category that happend during **weekdays**

In [48]:
df_sf = pd.read_csv("df_sf_robbery.csv")
df_sf.head()

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId
0,160009956,ROBBERY,ROBBERY OF A CHAIN STORE WITH BODILY FORCE,Monday,01/04/2016 12:00:00 AM,17:50,TARAVAL,"ARREST, BOOKED",3200 Block of 20TH AV,-122.475647,37.728528,"(37.7285280627465, -122.475647460786)",16000995603044
1,160014836,ROBBERY,ROBBERY OF A CHAIN STORE WITH A KNIFE,Wednesday,01/06/2016 12:00:00 AM,14:23,RICHMOND,NONE,700 Block of CABRILLO ST,-122.466559,37.775192,"(37.7751918267217, -122.466558780683)",16001483603042
2,160015486,ROBBERY,"ROBBERY, BODILY FORCE",Wednesday,01/06/2016 12:00:00 AM,17:20,MISSION,JUVENILE BOOKED,CAPP ST / 22ND ST,-122.417661,37.755504,"(37.7555043420028, -122.417660640341)",16001548603074
3,160016951,ROBBERY,"ROBBERY ON THE STREET, STRONGARM",Thursday,01/07/2016 12:00:00 AM,02:00,MISSION,NONE,CASTRO ST / 17TH ST,-122.435188,37.762670,"(37.7626702770872, -122.435187699349)",16001695103014
4,160016989,ROBBERY,ATTEMPTED ROBBERY WITH BODILY FORCE,Thursday,01/07/2016 12:00:00 AM,10:55,SOUTHERN,"ARREST, BOOKED",5TH ST / TEHAMA ST,-122.404408,37.781076,"(37.7810759636856, -122.404407887961)",16001698903474


<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #ffffff; color: navy;">Map1: folium.Marker()  &nbsp;&nbsp;&nbsp;&nbsp;</h1> 

We add markers to the map with folium.Marker(). Each marker below corresponds to a different robbery.

In [49]:
# Create a map
map1_sf = folium.Map(location=[37.78363773158004, -122.40941186660926], tiles='cartodbpositron', zoom_start=13)

# Add points to the map
for idx, row in df_sf.iterrows():
    Marker([row['Y'], row['X']]).add_to(map1_sf)

# Display the map
map1_sf

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #ffffff; color: navy;">Map2: folium.plugins.MarkerCluster()  &nbsp;&nbsp;&nbsp;&nbsp;</h1> 

If we have a lot of markers to add, `folium.plugins.MarkerCluster()` can help to declutter the map.  Each marker is added to a `MarkerCluster` object.

In [50]:
# Create the map
map2_sf = folium.Map(location=[37.78363773158004, -122.40941186660926], tiles='cartodbpositron', zoom_start=13)

# Add points to the map
mc = MarkerCluster()
for idx, row in df_sf.iterrows():
    if not math.isnan(row['X']) and not math.isnan(row['Y']):
        mc.add_child(Marker([row['Y'], row['X']]))
map2_sf.add_child(mc)

# Display the map
map2_sf

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #ffffff; color: navy;">Map3: Heatmap with folium.plugins.HeatMap()  &nbsp;&nbsp;&nbsp;&nbsp;</h1> 

 This shows the density of crime in different areas of the city, where yellow areas have relatively more criminal incidents.

In [51]:
# Create a base map
map3_sf = folium.Map(location=[37.78363773158004, -122.40941186660926], tiles='cartodbpositron', zoom_start=12)

# Add a heatmap to the base map
HeatMap(data=df_sf[['Y', 'X']], radius=10).add_to(map3_sf)

# Display the map
map3_sf